<a href="https://colab.research.google.com/github/RxnAch/DeepLearning/blob/main/Parameter_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Parameter Management:

In this section, we cover the following:
- Accessing parameters.
- Parameters Initialization.
- sharing parameters.

Note: **Why Initialize Weights?**

The aim of weight initialization is to prevent layer activation outputs from exploding or vanishing during the course of a forward pass through a deep neural network. If either occurs, loss gradients will either be too large or too small to flow backwards beneficially, and the network will take longer to converge, if it is even able to do so at all.

We start by focusing on an MLP with one hidden layer.

In [50]:
import torch 
import torch.nn as nn
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size = (2,4))
net(X)

tensor([[-0.3549],
        [-0.2534]], grad_fn=<AddmmBackward>)

#Parameter Access

Let us start with how to access parameters. When a model is defined via **Sequential** class, we can access any layer by indexing the model as it were a list.

In [8]:
print(net[0].state_dict()) 

OrderedDict([('weight', tensor([[-0.4871, -0.3918, -0.0052,  0.1247],
        [-0.3138,  0.3640, -0.2196,  0.2406],
        [ 0.4354,  0.4369,  0.4841,  0.1310],
        [-0.0352,  0.3481, -0.2443, -0.4267],
        [-0.4063, -0.3515, -0.1838,  0.3964],
        [ 0.1065,  0.0137, -0.0958,  0.1224],
        [ 0.3132, -0.4663,  0.4263, -0.0062],
        [-0.4908, -0.4490, -0.3703,  0.1133]])), ('bias', tensor([ 0.1719,  0.2837, -0.4237, -0.4036, -0.1499, -0.1344, -0.1332,  0.1167]))])


In [10]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2642, -0.2312, -0.2126, -0.3069, -0.2572,  0.0727,  0.2971,  0.0013]])), ('bias', tensor([-0.3293]))])


#Targeted Parameters

Note that each parameter is represented as an instance of the parameter class. To do anything useful with the parameters, we first need to access the underlying numerical values. There are several ways to do this. Some are simpler while others are more general. The following code extracts the bias from the second neural network layer, which returns a parameter class instance, and further accesses that parameter’s value.

In [13]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3293], requires_grad=True)
tensor([-0.3293])


Parameters are complex objects, containing values, gradients, and additional information. That’s why we need to request the value explicitly.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.

In [14]:
net[2].weight.grad == None 

True

#All Parameters at Once

When we need to perform operations on all parameters, accessing them one-by-one can grow tedious. The situation can grow especially unwieldy when we work with more complex blocks (e.g., nested blocks), since we would need to recurse through the entire tree to extract each sub-block’s parameters. Below we demonstrate accessing the parameters of the first fully-connected layer vs. accessing all layers.

In [22]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name ,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


This provides us with another way of accessing the parameters of the network as follows.

In [23]:
net.state_dict()['2.bias'].data

tensor([-0.3293])

#Collecting Parameters from Nested Blocks





In [28]:
def block1():
  return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
  net = nn.Sequential()
  for i in range(4):
    net.add_module(f'block{i}',block1())
  return net

rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)


tensor([[-0.0613],
        [-0.0613]], grad_fn=<AddmmBackward>)

Now that we have designed the network, let us see how it is organized.

In [29]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


Since the layers are hierarchically nested, we can also access them as though indexing through nested lists.

In [30]:
rgnet[0][1][0].bias.data

tensor([ 0.0554, -0.3469, -0.0904, -0.4319,  0.3093, -0.0400,  0.2955,  0.4666])

#Parameter Initialization
Now that we know how to access the parameters, let us look at how to initialize them properly.

Let us begin by calling on built-in initializers. The code below initializes all weight parameters as Gaussian random variables with standard deviation 0.01, while bias parameters cleared to zero.

In [34]:
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight,mean= 0 ,std=0.01)
    nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data,net[0].bias.data[0]

(tensor([[ 8.4527e-03,  1.5895e-03,  5.2931e-05, -4.4746e-03],
         [ 1.3336e-02, -2.7191e-03,  1.1000e-03,  1.4206e-02],
         [-8.2380e-03, -5.9766e-03,  1.8997e-02, -1.1486e-02],
         [ 3.4851e-03, -2.7606e-03,  1.5103e-02,  1.5207e-02],
         [ 5.7806e-03,  1.1359e-02, -3.9109e-03,  1.5646e-02],
         [ 1.5603e-03,  1.9837e-02,  1.6610e-02,  6.9621e-05],
         [-5.7602e-03,  5.6820e-03,  9.2778e-03, -7.6784e-03],
         [ 1.3881e-04, -1.4666e-02,  6.0874e-03,  5.7434e-03]]), tensor(0.))

We can also initialize all the parameters to a given constant value (say, 1).

In [36]:
def init_constant(m):#m for model
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight,1)
    nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data,net[0].bias.data

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]), tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

We can also apply different initializers for certain blocks. For example, below we initialize the first layer with the Xavier initializer and initialize the second layer to a constant value of 42.

In [40]:
def xavier(m):
  if type(m) == nn.Linear:
    nn.init.xavier_uniform_(m.weight)
def init_42(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight,42)
net[0].apply(xavier)
net[2].apply(init_42)

print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-0.5143, -0.3079, -0.1680, -0.5537],
        [ 0.3359,  0.2200,  0.5581,  0.2553],
        [-0.5112, -0.3524, -0.6932,  0.0339],
        [-0.5928,  0.5188, -0.0776, -0.1287],
        [-0.0163, -0.4150,  0.0600, -0.2100],
        [-0.0932,  0.1974, -0.3519, -0.1786],
        [-0.6719, -0.0134,  0.4952,  0.4949],
        [-0.1278, -0.3438,  0.0208, -0.4576]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [46]:
def my_init(m):
  if type(m)==nn.Linear:
    print("Init",*[(name,param.shape) for name,param in m.named_parameters()][0])
    nn.init.uniform_(m.weight,-10,10)
    m.weight.data *= m.weight.data.abs() >=5
net.apply(my_init)
net[0].weight


Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Parameter containing:
tensor([[-0.0000, -0.0000,  6.1151,  6.3900],
        [-0.0000, -0.0000,  0.0000, -0.0000],
        [ 0.0000,  8.5742, -6.2862, -9.9187],
        [ 0.0000, -6.2209,  7.1506, -7.6283],
        [ 9.5983,  7.0271,  0.0000, -0.0000],
        [-0.0000, -6.3133, -0.0000, -0.0000],
        [-8.4525, -8.7357,  0.0000, -6.6900],
        [-7.0515, -6.9889, -0.0000, -0.0000]], requires_grad=True)

In [48]:
net[0].weight.data[:] +=1
net[0].weight.data[0,0] = 42
net[0].weight.data

tensor([[42.0000,  2.0000,  8.1151,  8.3900],
        [ 2.0000,  2.0000,  2.0000,  2.0000],
        [ 2.0000, 10.5742, -4.2862, -7.9187],
        [ 2.0000, -4.2209,  9.1506, -5.6283],
        [11.5983,  9.0271,  2.0000,  2.0000],
        [ 2.0000, -4.3133,  2.0000,  2.0000],
        [-6.4525, -6.7357,  2.0000, -4.6900],
        [-5.0515, -4.9889,  2.0000,  2.0000]])

#Sharing Parameters

Often, we want to share parameters across multiple layers. Let us see how to do this elegantly

In [51]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
